In [2]:
import requests
import arcpy
import pandas as pd
from arcgis import features, GeoAccessor
from arcpy import Point, PointGeometry, SpatialReference
import os

In [3]:
directory = r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2.gdb"

# Weather Data

## Weather Data ETL

In [4]:
# API Call and Formatting
url= "https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&month=5&year=2024"
request = requests.get(url)
response = request.json()
df=pd.json_normalize(response['features'])


In [5]:
# Remove Nested Fields
raw_weather = pd.DataFrame.from_records(df)
raw_weather.columns = raw_weather.columns.str.replace("properties.", "", regex=True)

# Apply Geometry
raw_weather["longitude"] = raw_weather["geometry.coordinates"].apply(lambda x: x[0])  # Extract longitude
raw_weather["latitude"] = raw_weather["geometry.coordinates"].apply(lambda x: x[1])   # Extract latitude

In [6]:
# Create Copy of Dataframe with Filtered Columns
weather = raw_weather[["station", "date", "max_tmpf", "min_tmpf", "latitude", "longitude"]].copy()

## Weather Data QAQC

### Weather Station Geometry: Drop NA Values, Check Bounding Box

In [7]:
# Drop NA Values
weather = weather.dropna(subset=["latitude", "longitude"])

In [8]:
# MN Bounding Box
min_lat, max_lat = 43.4994, 49.3844
min_lon, max_lon = -97.2392, -89.4912

# Check Bounding Box using "in_minnesota" column
weather["in_minnesota"] = weather.apply(lambda row: min_lon <= row["longitude"] <= max_lon and min_lat <= row["latitude"] <= max_lat, axis=1)

# Filter Dataframe to Only Rows "in_minnesota"
mn_weather = weather[weather["in_minnesota"]]

### Temperature Values: Drop Obvious Outliers Using IQR

In [9]:
# Max Temp Outliers
# Set Upper and Lower Bounds
Q1_max = mn_weather['max_tmpf'].quantile(0.25)
Q3_max = mn_weather['max_tmpf'].quantile(0.75)
IQR_max = Q3_max - Q1_max

lower_bound_max = Q1_max - 1.5 * IQR_max
upper_bound_max = Q3_max + 1.5 * IQR_max

# Identify Outliers
outliers_max = mn_weather[(mn_weather['max_tmpf'] < lower_bound_max) | (mn_weather['max_tmpf'] > upper_bound_max)]

# Remove Outliers
mn_weather = mn_weather[(mn_weather['max_tmpf'] >= lower_bound_max) & (mn_weather['max_tmpf'] <= upper_bound_max)]

In [10]:
# Min Temp Outliers
# Set Upper and Lower Bounds
Q1_min = mn_weather['min_tmpf'].quantile(0.25)
Q3_min = mn_weather['min_tmpf'].quantile(0.75)
IQR_min = Q3_min - Q1_min

lower_bound_min = Q1_min - 1.5 * IQR_min
upper_bound_min = Q3_min + 1.5 * IQR_min

# Identify Outliers
outliers_min = mn_weather[(mn_weather['min_tmpf'] < lower_bound_min) | (mn_weather['min_tmpf'] > upper_bound_min)]

# Remove Outliers
mn_weather = mn_weather[(mn_weather['min_tmpf'] >= lower_bound_min) & (mn_weather['min_tmpf'] <= upper_bound_min)]

In [11]:
# QA Result
mn_weather

,station,date,max_tmpf,min_tmpf,latitude,longitude,in_minnesota
0,MN001,2024-05-01,64.219980,46.940020,43.508331,-93.354057,True
1,MN002,2024-05-01,60.440020,45.680000,44.906800,-94.119100,True
2,MN003,2024-05-01,64.580000,46.940020,44.478500,-93.292427,True
3,MN004,2024-05-01,61.160010,48.199990,45.642921,-92.992752,True
4,MN005,2024-05-01,61.160010,44.960010,46.212570,-92.838562,True
...,...,...,...,...,...,...,...
5048,MN168,2024-05-31,71.419975,47.480000,48.195351,-96.891144,True
5049,MN169,2024-05-31,70.880000,49.099990,48.647079,-96.380699,True
5050,MN170,2024-05-31,70.519980,54.860012,46.524620,-94.296143,True
5051,MN171,2024-05-31,73.760010,51.080000,46.771381,-92.137520,True


## Convert Weather Data to Feature Class

In [12]:
# Convert to Spatially Enabled DataFrame
sedf=GeoAccessor.from_xy(mn_weather, 'longitude', 'latitude')
out_name = "weather_fc"

#Convert SEDF to Feature Class
sedf.spatial.to_featureclass(location=f"{directory}/{out_name}")

'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\Lab2\\Lab2.gdb\\weather_fc'

# Elevation and Land Cover Data

In [82]:
dem_raw= "C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\MGC_30m_DEM\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m"
landcover_raw= "C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\MN_NLCD_2019\MN_NLCD_2019_Land_Cover.tif"

## Raster QAQC

In [37]:
# Check For Blank/Null Values
def check_nodata(in_raster):
    
    # Store Output 
    no_data = arcpy.management.GetRasterProperties(in_raster, "ANYNODATA").getOutput(0)
    
    if no_data == "1":
        print("There are null values")
        
    else:
        print("There are NOT any null values")
    
check_nodata(dem_raw)
check_nodata(landcover_raw)

There are null values
There are null values


In [74]:
# Check SRS
def check_sr(in_raster, in_sr):
    sr = arcpy.Describe(in_raster).spatialReference
    in_sr_code = arcpy.SpatialReference(in_sr)
    
    if sr.factoryCode == in_sr_code.factoryCode:
        print("Coordinate Systems Match")
        
    else:
        print("Coordinate Systems Do Not Match")
        print(f"Actual Coordinate System is: {sr.factoryCode}")

check_sr(dem_raw, 26915)
check_sr(landcover_raw, 26915)

Coordinate Systems Match
Coordinate Systems Match


In [79]:
# Change Bounding Box to 26915 Coordinate Values
# Define the bounding box in EPSG:4326
bbox_4326 = [(-97.5, 43.0), (-89.0, 49.5)]

# Create a spatial reference object for EPSG:26915
sr_26915 = SpatialReference(26915)

# Transform the bounding box coordinates to EPSG:26915
transformed_bbox = []
for x, y in bbox_4326:
    point = PointGeometry(Point(x, y), SpatialReference(4326))
    point_projected = point.projectAs(sr_26915)
    transformed_bbox.append((point_projected.firstPoint.X, point_projected.firstPoint.Y))

# Use the transformed bounding box in the check_box function
xmin, ymin = transformed_bbox[0]
xmax, ymax = transformed_bbox[1]
transformed_bbox

[(133186.3828544069, 4770648.492016879), (789594.190423549, 5490732.38301064)]

In [84]:
# Check Bounding Box
def check_box(in_raster, ymin, ymax, xmin, xmax):
    
    left = float(arcpy.management.GetRasterProperties(in_raster, "LEFT").getOutput(0))
    right = float(arcpy.management.GetRasterProperties(in_raster,"RIGHT").getOutput(0))
    top = float(arcpy.management.GetRasterProperties(in_raster, "TOP").getOutput(0))
    bottom = float(arcpy.management.GetRasterProperties(in_raster,"BOTTOM").getOutput(0))
    
    print(left, right , top , bottom)
    
    if left < xmin or right > xmax or top > ymax or bottom < ymin:
        print("Raster is not Within Bounding Box")
    
    else: 
        print("Raster is Within Bounding Box")
        
check_box(landcover_raw, 4770648.492016879, 5490732.38301064, 133186.3828544069, 789594.190423549)
check_box(dem_raw, 4770648.492016879, 5490732.38301064, 133186.3828544069, 789594.190423549)

-112390.0 801260.0 5566390.0 4600900.0
Raster is not Within Bounding Box
189775.332039 761665.332039 5472435.370038 4816305.370038
Raster is Within Bounding Box


### Fixing and Exporting Rasters

In [85]:
# Clip and Export Raster outside Bounding Box
output_raster = f"{directory}/landcover"
arcpy.management.Clip(landcover_raw, "133186.382854406 4770648.492016879 789594.190423549 5490732.38301064", output_raster)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\Lab2\\Lab2.gdb\\landcover'>

In [91]:
# Export Raster
arcpy.conversion.RasterToGeodatabase(dem_raw, directory)

<Result ''>

# Roads Data QAQC

In [89]:
selection = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="MnDOT_Roadway_Routes_in_Minnesota",
    selection_type="NEW_SELECTION",
    where_clause="ROUTE_SI_1 = 'County' Or ROUTE_SI_1 = 'Interstate' Or ROUTE_SI_1 = 'State' Or ROUTE_SI_1 = 'U.S.'",
    invert_where_clause=None
)

In [90]:
arcpy.conversion.ExportFeatures(
    in_features="MnDOT_Roadway_Routes_in_Minnesota",
    out_features=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2.gdb\major_roads_raw"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\Lab2\\Lab2.gdb\\major_roads_raw'>

In [92]:
arcpy.gapro.ClipLayer(
    input_layer="major_roads_raw",
    clip_layer=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2.gdb\state_of_minnesota",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2.gdb\major_roads_fv"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\Lab2\\Lab2.gdb\\major_roads_fv'>

In [14]:
# Export to PostGIS database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="weather_fc;major_roads_fv",
    Output_Geodatabase=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2PostGIS.sde"
)


<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\Lab2\\Lab2PostGIS.sde'>

In [ ]:
# Raster to Postgres Database
arcpy.conversion.RasterToGeodatabase(
    f'{os.path.join(directory, "landcover")};{os.path.join(directory, "digital_elevation_model_30m")}',
    pgdb
)